In [1]:
# Load essential libraries
import pormake
from pormake import *
from pormake.experimental.decomposer import MOFDecomposer

import ase

import rodmof
from rodmof import *
from rodmof.rod_utils import *

# How to find unique topology for wine-rack MOFs

1. Use subgraph to identify building blocks (rod and edge) from raw cif files. -> Can collect indices for node and edge building blocks from give cif file 
2. Refine node_bb list manually (the list obtained from Step 1 usually contain overlapped part because of repeating characteristics of rod building blocks)
3. Add rod metal index to edge_bb list to insert pseudo-edge between rod building block to find coordination sequence.
4. Scratch bond information between metal rod and edges
5. Add (metal idx, metal idx) to step 4 to endow pseudo bonds between rod building blocks
6. Get coordination sequence 

* Step 1 and 4 were used using decomposer module in PORMAKE library
* Ase gui was used to manually investigae atom indices (highly recommend to use view() function in ase.visualize module)
* raw cif files used in this work were prepared in the "assets" directory

## 1. Find cds

coordination sequence:</br>
vertex 1: 4, 12, 30, 58, 94, 138, 190, 250, 318, 394

* Ref. - https://rcsr.anu.edu.au/nets/cds

In [2]:
cif_path = "../assets/Co_bdp.cif"

In [3]:
# load MOFDecomposer module to collect indices of node building block and edge building block (Step 1)
decomposer = MOFDecomposer(cif=cif_path)
decomposer.cleanup()
bbs_indices = decomposer.building_block_atom_indices
connect_sites = decomposer.connecting_sites

/home/lim/anaconda3/envs/test/lib/python3.8/site-packages/ase/io/cif.py:401: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  warnings.warn(


In [4]:
# node bbs have > 2 connecting site
# edge bbs have only 2 connecting site
raw_node_indices, raw_edge_indices = [], []

for bbs_index in bbs_indices:
    n_connect = 0
    for ind in bbs_index:
        if ind in connect_sites:
            n_connect+=1
    
    raw_edge_indices.append(bbs_index) if n_connect == 2 else raw_node_indices.append(bbs_index)

In [5]:
print('Nodes')
for i, l in enumerate(raw_node_indices):
    print("Indices of raw node {}: {}".format(i, l))

print('Edges')
for i, l in enumerate(raw_edge_indices):
    print("Indices of raw edge {}: {}".format(i, l))

Nodes
Indices of raw node 0: {0, 1, 2, 6, 7, 8, 9, 24, 25, 26, 30, 31, 32, 33, 48, 49, 50, 54, 55, 56, 57, 72, 73, 74, 78, 79, 80, 81, 97, 99}
Indices of raw node 1: {12, 13, 14, 18, 19, 20, 21, 36, 37, 38, 42, 43, 44, 45, 60, 61, 62, 66, 67, 68, 69, 84, 85, 86, 90, 91, 92, 93, 96, 98}
Edges
Indices of raw edge 0: {64, 65, 3, 4, 5, 70, 71, 10, 11, 63}
Indices of raw edge 1: {15, 16, 17, 51, 52, 53, 22, 23, 58, 59}
Indices of raw edge 2: {34, 35, 87, 88, 89, 27, 28, 29, 94, 95}
Indices of raw edge 3: {39, 40, 41, 75, 76, 77, 46, 47, 82, 83}


In [6]:
# indices can be easily visualized or identified using view() module of ase.visualize

# node indices were manually collected (Step 2)
_node_bb =[
    {0, 1, 2, 6, 7, 8, 9, 48, 49, 50, 54, 55, 56, 57, 97, 99},
    {24, 25, 26, 30, 31, 32, 33, 72, 73, 74, 78, 79, 80, 81, 97, 99},
    {36, 37, 38, 42, 43, 44, 45, 84, 85, 86, 90, 91, 92, 93, 96, 98},
    {12, 13, 14, 18, 19, 20, 21, 60, 61, 62, 66, 67, 68, 69, 96, 98},
]

# rod metal indices: 96, 97, 98, 99 (Step 3)
_edge_bb = [
    {3, 4, 5, 10, 11, 63, 64, 65, 70, 71},
    {15, 16, 17, 22, 23, 51, 52, 53, 58, 59},
    {27, 28, 29, 34, 35, 87, 88, 89, 94, 95},
    {39, 40, 41, 46, 47, 75, 76, 77, 82, 83},
    {96}, {97}, {98}, {99}, # rod metal indices
]

In [7]:
# Collect bond information (Step 4)
raw_bond = decomposer.connecting_bonds

print('Bonds')
for i, l in enumerate(raw_bond):
    print("Bond {}: {}".format(i, l))

Bonds
Bond 0: (2, 3)
Bond 1: (62, 63)
Bond 2: (50, 51)
Bond 3: (14, 15)
Bond 4: (26, 27)
Bond 5: (86, 87)
Bond 6: (74, 75)
Bond 7: (38, 39)


In [8]:
# Add (metal idx, metal idx) as bond site (Step 5)
_bond = [(2, 3), (62, 63), (50, 51), (14, 15), (26, 27), (86, 87), (74, 75), (38, 39), 
         (97, 97), (99, 99), (96, 96), (98, 98)]

In [9]:
# Get coordination sequence (Step 6)
cobdp = ase.io.read(cif_path)
rod_cobdp = rod_mof.RodMOF(cobdp, _node_bb, _edge_bb, _bond)
cobdp_coord_seq = rod_cobdp.get_coord_seq()

for i, l in enumerate(cobdp_coord_seq):
    print('Coord. seq. for vertex {}: {}'.format(i, l))

Coord. seq. for vertex 0: [  4  12  30  58  94 138 190 250 318 394]


## 2. Find dia

coordination sequence:</br>
vertex 1: 4, 12, 24, 42, 64, 92, 124, 162, 204, 252

* Ref. - https://rcsr.anu.edu.au/nets/dia

In [13]:
cif_path = "../assets/MIL53.cif"

In [14]:
# load MOFDecomposer module to collect indices of node building block and edge building block (Step 1)
decomposer = MOFDecomposer(cif=cif_path)
decomposer.cleanup()
bbs_indices = decomposer.building_block_atom_indices
connect_sites = decomposer.connecting_sites

In [15]:
# node bbs have > 2 connecting site
# edge bbs have only 2 connecting site
raw_node_indices, raw_edge_indices = [], []

for bbs_index in bbs_indices:
    n_connect = 0
    for ind in bbs_index:
        if ind in connect_sites:
            n_connect+=1
    
    raw_edge_indices.append(bbs_index) if n_connect == 2 else raw_node_indices.append(bbs_index)

In [16]:
print('Nodes')
for i, l in enumerate(raw_node_indices):
    print("Indices of raw node {}: {}".format(i, l))

print('Edges')
for i, l in enumerate(raw_edge_indices):
    print("Indices of raw edge {}: {}".format(i, l))

Nodes
Indices of raw node 0: {0, 65, 64, 36, 68, 7, 72, 43, 12, 48, 50, 19, 52, 53, 56, 24, 60, 31}
Indices of raw node 1: {1, 66, 37, 6, 70, 71, 74, 42, 13, 49, 18, 51, 54, 30, 25, 58, 59, 62}
Edges
Indices of raw edge 0: {2, 3, 4, 5, 38, 39, 40, 41, 55, 57}
Indices of raw edge 1: {8, 9, 10, 11, 44, 45, 46, 47, 61, 63}
Indices of raw edge 2: {67, 69, 14, 15, 16, 17, 26, 27, 28, 29}
Indices of raw edge 3: {32, 33, 34, 35, 73, 75, 20, 21, 22, 23}


In [17]:
# indices can be easily visualized or identified using view() module of ase.visualize

# node indices were manually collected (Step 2)
_node_bb = [
    {1, 18, 30, 37, 49, 51, 54, 74, 70, 71},
    {6, 13, 25, 42, 49, 51, 58, 59, 62, 66},
    {7, 12, 24, 43, 48, 50, 60, 64, 65, 68},
    {0, 19, 31, 36, 48, 50, 56, 52, 53, 72}
]

# rod metal indices: 48, 49, 50, 51 (Step 3)
_edge_bb = [
    {2,  3,  4,  5, 38, 39, 40, 41, 55, 57},
    {8,  9, 10, 11, 44, 45, 46, 47, 61, 63},
    {14, 15, 16, 17, 26, 27, 28, 29, 67, 69},
    {20, 21, 22, 23, 32, 33, 34, 35, 73, 75},
    {48}, {49}, {50}, {51}, # rod metal indices
]

In [18]:
# Collect bond information (Step 4)
raw_bond = decomposer.connecting_bonds

print('Bonds')
for i, l in enumerate(raw_bond):
    print("Bond {}: {}".format(i, l))

Bonds
Bond 0: (56, 57)
Bond 1: (54, 55)
Bond 2: (60, 61)
Bond 3: (62, 63)
Bond 4: (66, 67)
Bond 5: (68, 69)
Bond 6: (72, 73)
Bond 7: (74, 75)


In [19]:
# Add (metal idx, metal idx) as bond site (Step 5)
_bond = [(56, 57), (54, 55), (60, 61), (62, 63), (66, 67), (68, 69), (72, 73), (74, 75),
         (48, 48), (49, 49), (50, 50), (51, 51)]

In [20]:
# Get coordination sequence (Step 6)
mil53 = ase.io.read(cif_path)
rod_mil53 = rod_mof.RodMOF(mil53, _node_bb, _edge_bb, _bond)
mil53_coord_seq = rod_mil53.get_coord_seq()

for i, l in enumerate(mil53_coord_seq):
    print('Coord. seq. for vertex {}: {}'.format(i, l))

Coord. seq. for vertex 0: [  4  12  24  42  64  92 124 162 204 252]


## 3. Find mog

coordination sequence:</br>
vertex 1: 4, 10, 24, 40, 64, 90, 120, 160, 200, 244</br>
vertex 2: 4, 11, 24, 41, 62, 90, 122, 157, 200, 247

* Notice: MIL118 does not have edge building blocks, so connecting bonds were regarded as alternative indices for edge building blocks

Ref. - https://rcsr.anu.edu.au/nets/mog

In [23]:
cif_path = "../assets/MIL118.cif"

In [24]:
# load MOFDecomposer module to collect indices of node building block and edge building block (Step 1)
decomposer = MOFDecomposer(cif=cif_path)
decomposer.cleanup()
bbs_indices = decomposer.building_block_atom_indices
connect_sites = decomposer.connecting_sites

In [25]:
# node bbs have > 2 connecting site
# edge bbs have only 2 connecting site
raw_node_indices, raw_edge_indices = [], []

for bbs_index in bbs_indices:
    n_connect = 0
    for ind in bbs_index:
        if ind in connect_sites:
            n_connect+=1
    
    raw_edge_indices.append(bbs_index) if n_connect == 2 else raw_node_indices.append(bbs_index)

In [26]:
print('Nodes')
for i, l in enumerate(raw_node_indices):
    print("Indices of raw node {}: {}".format(i, l))

print('Edges')
for i, l in enumerate(raw_edge_indices):
    print("Indices of raw edge {}: {}".format(i, l))

Nodes
Indices of raw node 0: {0, 1, 32, 3, 33, 37, 36, 8, 9, 11, 44, 46, 20, 21, 23, 28, 29, 31}
Indices of raw node 1: {2, 6, 40, 41, 18, 51, 50, 22}
Indices of raw node 2: {34, 35, 4, 5, 38, 7, 39, 12, 45, 13, 47, 16, 17, 15, 19, 24, 25, 27}
Indices of raw node 3: {10, 43, 42, 14, 48, 49, 26, 30}
Edges


In [27]:
raw_edge = decomposer.connecting_bonds

print('Alternative edge indices')
for i, l in enumerate(raw_edge):
    print("Edge {}: {}".format(i, l))

Alternative edge indices
Edge 0: (2, 3)
Edge 1: (6, 7)
Edge 2: (18, 19)
Edge 3: (22, 23)
Edge 4: (26, 27)
Edge 5: (30, 31)
Edge 6: (10, 11)
Edge 7: (14, 15)


In [28]:
# indices can be easily visualized or identified using view() module of ase.visualize

# node indices were manually collected (Step 2)
_node_bb = [
    {5, 12, 15, 17, 24, 27, 34, 35, 38, 45},
    {4,  7, 13, 16, 19, 25, 34, 38, 39, 47},
    {1,  8, 11, 21, 28, 31, 32, 33, 36, 44},
    {0,  3,  9, 20, 23, 29, 32, 36, 37, 46},
    {2,  6, 18, 22, 40, 41, 50, 51},
    {10, 14, 26, 30, 42, 43, 48, 49}
]

# rod metal indices: 32, 34, 36, 38 (Step 3)
_edge_bb = [{26, 27}, {30, 31}, {10, 11}, {14, 15}, {6, 7}, {18, 19}, {2, 3}, {22, 23},
    {32}, {34}, {36}, {38}, # rod metal indices
]

In [29]:
# All atoms of edge building blocks were treated similar to rod metal atom
# For example, considering that (26, 27) is in _edge_bb, (26, 26) and (27, 27) should be involved in _bond
_bond = [(30, 30), (26, 26), (10, 10), (14, 14), (2, 2), (22, 22), (6, 6), (18, 18),
        (31, 31), (27, 27), (11, 11), (15, 15), (3, 3), (23, 23), (7, 7), (19, 19),
        (32, 32), (34, 34), (36, 36), (38, 38)]

In [30]:
# Get coordination sequence (Step 6)
mil118 = ase.io.read(cif_path)
rod_mil118 = rod_mof.RodMOF(mil118, _node_bb, _edge_bb, _bond)
mil118_coord_seq = rod_mil118.get_coord_seq()

for i, l in enumerate(mil118_coord_seq):
    print('Coord. seq. for vertex {}: {}'.format(i, l))

Coord. seq. for vertex 0: [  4  10  24  40  64  90 120 160 200 244]
Coord. seq. for vertex 1: [  4  11  24  41  62  90 122 157 200 247]
